# CW-Husky Triggers

This notebook shows how to use Husky's different trigger modules.

In [ ]:
SCOPE="OPENADC"
PLATFORM = 'CW308_SAM4S'
#PLATFORM = 'CW308_STM32F3'

In [ ]:
import chipwhisperer as cw
%run "../../Setup_Scripts/Setup_Generic.ipynb"

This notebook should run on all simpleserial targets, but ADC triggering works better on this target firmware, on our STM32 target. (Unfortunately ADC triggering doesn't work very well on the SAM4S target because there is not as much distinction in the power trace between idle and busy.)

Here we use a pre-compiled target firmware because the SAD parameters in particular are very much tied to the firmware.

If you use a different target and/or firmware, you will need to adjust the triggering thresholds for the ADC and SAD sections.

In [ ]:
cw.program_target(scope, prog, "../../../firmware/mcu/simpleserial-trace/simpleserial-trace-{}.hex".format(PLATFORM))

In [ ]:
reset_target(scope)

## 1. UART Triggering

It's possible to trigger on the UART data that is sent to or received from the target.

In [ ]:
scope.trigger.module = 'UART'
scope.UARTTrigger.enabled = True
scope.UARTTrigger.baud = 38400
scope.gain.db = 12

We set up two matching rules: rule 0 matches the plaintext that we'll send to the target on tio2, and rule 1 matches the response that the target will send back on tio1. Then we demonstrate triggering on each.

The match patterns can be up to 8 bytes long. There is a bit-wise mask parameter which defaults to all ones. Up to 8 rules can be specified and selectively enabled.

In [ ]:
scope.UARTTrigger.set_pattern_match(0, 'p00') # match 'p'... that we send
scope.UARTTrigger.set_pattern_match(1, 'r7DF7') # match 'r'... that we receive

Let's first trigger on the plaintext command that's sent to the target on tio2.

The UART trigger module can use any trigger input that you can specify to the normal trigger module.

If you have an oscilloscope or logic analyzer, probe tio2, tio1, and Trigger/Glitch Out MCX. The MCX is the internal ADC capture trigger. You use this to confirm that the trigger event occurs when expected.

The trigger fires after the last pattern byte is received (i.e. in the case of the 'p00' pattern, after the second '0'). *This is different behaviour from earlier ChipWhisperer releases (5.7.0 and earlier), where the trigger would fire after 8 bytes are received (i.e. in the 'p00' example, it would fire after receiving 'p00' following by 5 "don't care" bytes).*

In some situations you may wish for the trigger to fire later; for example if you want the trigger to fire after receiving any 8-byte pattern which starts with 'p0'. In this situation, specify the pattern and mask like this:

```python
scope.UARTTrigger.set_pattern_match(0, 'p0xxxxxx', mask=[255,255,0,0,0,0,0,0])
```


In [ ]:
scope.trigger.triggers = 'tio2'
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0,1]
scope.io.glitch_trig_mcx = 'trigger'

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

The UARTTrigger module records the actual UART data that caused the last trigger (useful in the case of wildcards), and a counter for each time a rule matched and triggered:

In [ ]:
scope.UARTTrigger.matched_pattern_data(as_string=True)

In [ ]:
scope.UARTTrigger.matched_pattern_counts

Now let's switch over to triggering on the response on tio1. Again, we must specify on which pin the trigger module should use, and which rule should cause a trigger:

In [ ]:
scope.trigger.triggers = 'tio1'
scope.UARTTrigger.trigger_source = 1
scope.UARTTrigger.rules_enabled = [0,1]

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
scope.UARTTrigger.matched_pattern_data(as_string=True)

In [ ]:
scope.UARTTrigger.matched_pattern_counts

`scope.UARTTrigger` actually runs on the same FPGA logic as `scope.trace` aka TraceWhisperer (the ARM trace sniffer), because ARM trace in SWO mode is just UART.

This means it's also possible to record all the UART data. Since the data is time-stamped (exactly like it is in TraceWhisperer), how much data can be recorded depends on its nature. There is space for up to 8192 bytes, but periods of inactivity will require some of that capacity to be used for additional timestamp storage.

(If you're interested in using the timestamp data, the [TraceWhisperer.ipynb](https://github.com/newaetech/DesignStartTrace/blob/master/jupyter/TraceWhisperer.ipynb) notebook will show you how.)

By triggering on `tio1 and tio2`, we'll capture both the Tx and Rx traffic:

In [ ]:
scope.trigger.triggers = 'tio1 and tio2'
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0]

In [ ]:
scope.trace.trace_mode = 'swo'
scope.trace.capture.mode = 'count_writes'
scope.trace.capture.count = 0

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
raw = scope.trace.read_capture_data()

In [ ]:
len(raw)

`raw` is time-stamped raw UART data. There's a convenience method to decode it to ASCII:

In [ ]:
ascii = scope.UARTTrigger.uart_data(raw)
for b in ascii:
    print(b, end='')

The UART trigger module supports issuing multiple triggers: *up to* `scope.UARTTrigger.capture.max_triggers` can be issued. Once this limit is reached, no more triggers are issued until the scope is re-armed.

Finally, while these examples have used the usual tio1 and tio2 lines as inputs, you can use anything input that `scope.trigger.triggers` supports.

By default, the `UARTTrigger` module is set for 8-N-1 operation, since these are the UART settings for ChipWhisperer targets.

If you have a different target, you can adjust `scope.UARTTrigger.data_bits`, `scope.UARTTrigger.parity` and `scope.UARTTrigger.stop_bits` accordingly.

If `data_bits` is not 8, the pattern and mask are to be provided as a list of `data_bits`-sized integers.

If `parity` is used, the `scope.UARTTrigger.accept_parity_errors` provides the option of ignoring words with parity errors. When `scope.UARTTrigger.accept_parity_errors = True`, words with parity errors are accepted and pattern matching continues as though these words were received without error; when `scope.UARTTrigger.accept_parity_errors = False`, words with parity errors are ignored, as if they were never sent.

## 2. ADC level triggering

This one is a lot more simple: it simply triggers when ADC samples exceed a defined level.

This can be useful when the target is idle and "quiet" outside of the target operation.

We start by getting another power trace from "normal" triggering, and with proper gain and ADC settings, so we can observe how high the power samples go.

In [ ]:
scope.gain.db = 12
scope.adc.samples = 50000
scope.adc.presamples = 10000
scope.trigger.module = 'basic'
scope.trigger.triggers = 'tio4'

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
import numpy as np
output_notebook(INLINE)

o = figure(width=1200)

xrange = list(range(len(trace.wave)))
O = o.line(xrange, trace.wave)

In [ ]:
show(o)

If you're using the simpleserial-aes firmware on the STM32 target, you should see a large negative spike shortly after the start of the capture, which should be a good marker for level-based triggering:

(On the SAM4S, there isn't much difference between the power when idle vs encrypting, so the ADC trigger isn't as useful there.)

In [ ]:
scope.trigger.module = 'ADC'
scope.trigger.level = min(trace.wave)*0.95

In [ ]:
scope.adc.presamples = int(np.where(trace.wave == min(trace.wave[:20000]))[0][0])+ 5

In [ ]:
adc_triggered_trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
o = figure(width=1200)

O1 = o.line(xrange, trace.wave, line_color='black')
O2 = o.line(xrange, adc_triggered_trace.wave, line_color='blue')
O3 = o.line(xrange, abs(trace.wave - adc_triggered_trace.wave), line_color='red')

o.renderers.extend([Span(location=scope.trigger.level, dimension='width', line_color='green', line_width=2, line_dash='dashed')])

In [ ]:
show(o)

The two traces should be coincident or very nearly so.

With ADC-level triggering, a single trigger is issued: after the first trigger, no further triggers are issued, even if ADC samples exceed the level threshold, until the scope is re-armed.

# 3. SAD Triggering

Sum-of-Absolute Differences (SAD) has grown to have enough features to require its own notebook: [06 - Husky SAD Triggering.ipynb](06%20-%20Husky%20SAD%20Triggering.ipynb).

# 4. TraceWhisperer

Triggering from traces is covered in a separate notebook: [TraceWhisperer.ipynb](https://github.com/newaetech/DesignStartTrace/blob/master/jupyter/TraceWhisperer.ipynb).

# 5. Edge Count Triggering

This counts rising and falling edges of an input signal and triggers after a specified number of edges.

This can be very useful when you want to trigger from some protocol that Husky can't decode.

Here we'll demonstrate with UART because it's what we have with our targets, but it can be used for much more than that.

The input signal to the edge counter can be anything that `scope.trigger.triggers` supports (which, in addition to what you could use on CW-Lite/Pro, includes the USERIO data pins).

In [ ]:
scope.default_setup()

In [ ]:
scope.trigger.module = 'edge_counter'
scope.trigger.triggers = 'tio1'
#scope.trigger.triggers = 'tio2'
scope.trigger.edges = 3

We'll use Husky's logic analyzer to visualize the generated trigger. The trigger is a narrow single-cycle pulse of the ADC sampling clock, so we must sample fairly fast, which means we won't see all of the UART traffic.

In [ ]:
scope.trace.enabled = False
scope.LA.enabled = True
scope.LA.clk_source = 'pll'
scope.LA.oversampling_factor = 4
scope.LA.downsample = 1
scope.LA.capture_group = 'CW 20-pin'
scope.LA.trigger_source = 'falling_tio1'
#scope.LA.trigger_source = 'falling_tio2'
scope.LA.capture_depth = 16000
scope.adc.clip_errors_disabled = True

This is so that the internal trigger can be captured by `scope.LA`:

In [ ]:
scope.io.glitch_trig_mcx = 'trigger'

In [ ]:
scope.LA.arm()
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
assert not scope.LA.fifo_empty()
raw = scope.LA.read_capture_data()
tio1 = scope.LA.extract(raw, 0)
tio2 = scope.LA.extract(raw, 1)
trig = scope.LA.extract(raw, 7)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(width=1800)

xrange = list(range(len(tio1)))
O1 = o.line(xrange, tio1 + 4, line_color='black')
O2 = o.line(xrange, tio2 + 2, line_color='blue')
O3 = o.line(xrange, trig + 0, line_color='red')

legend = Legend(items=[
    LegendItem(label='tio1', renderers=[O1]),
    LegendItem(label='tio2', renderers=[O2]),
    LegendItem(label='internal capture trigger', renderers=[O3]),
])
o.add_layout(legend)

In [ ]:
show(o)

The first falling edge of TIO1 is missed by the `scope.LA` capture (because that's the edge which kicks off the capture).

Accounting for that, you can see that the trigger occurs immediately after the third edge of TIO1.

You can change `scope.trigger.edges` and see the trigger move around accordingly.

If you set it higher than the number of edges that are observed, the capture will time out (because no trigger was generated). If this happens, `scope.trigger.edges_seen` will show how many edges were observed.

In [ ]:
scope.trigger.edges_seen

Try the other trigger sources (don't forget to adjust `scope.LA.trigger_source` accordingly if you want to see the results via `scope.LA` as above).

## 6. Sequenced Triggers

See [04 - Husky Trigger Sequencer.ipynb](04%20-%20Husky%20Trigger%20Sequencer.ipynb) to learn how to sequencer multiple trigger events.